# Final Project: Global Football Data Analysis

This is the final project for the course **"Programming for Data Science"** at the **Faculty of Information Technology**, **University of Science, Vietnam National University, Ho Chi Minh City**.

### Project Overview

The project focuses on analyzing global football data to extract insights and patterns related to team performances, player statistics, and match outcomes.


### GitHub Repository

You can access the full project on GitHub:  
[Global Football Data Analysis Repository](https://github.com/trgtanhh04/Programming4DS_Final)

---

Instructor: Phạm Trọng Nghĩa  

Course: Programming for Data Science  

Date: 12/2024

---

## Import

In [1]:
import requests
import numpy as np
import pandas as pd
from typing import List
import os
import seaborn as sns
import matplotlib.pyplot as plt
import urllib.robotparser

---

## Data collecting

### What is the subject of your data?

The data in this project is about **international football results**, covering match outcomes, teams, goals, and match dates.

### What is the source of your data?

The data is sourced from Kaggle, available at:
[International Football Results Dataset](https://www.kaggle.com/datasets/martj42/international-football-results-from-1872-to-2017?select=shootouts.csv)


### Do the authors of this data allow you to use it like this?

The authors of this data allow its use under the **Open Data License** (or other specific licenses depending on the data source). You can check the data license at the following link:
- [Data License from Kaggle](https://creativecommons.org/publicdomain/zero/1.0/)

### How did the authors collect the data?

The data was collected using the following methods:
- **Web scraping**: Collecting data from reputable sports websites.
- **API**: Using public APIs from sports platforms to retrieve data.
- **Surveys**: Gathering information from surveys and research reports on football.

---

## Exploring data

### Read raw data from file

In [2]:
# YOUR CODE HERE
raw_goal_scorers = pd.read_csv("Data/goalscorers.csv")
raw_results = pd.read_csv("Data/results.csv")
raw_shootouts = pd.read_csv("Data/shootouts.csv")

In [3]:
raw_goal_scorers.head()

,date,home_team,away_team,team,scorer,minute,own_goal,penalty
0,1916-07-02,Chile,Uruguay,Uruguay,José Piendibene,44.0,False,False
1,1916-07-02,Chile,Uruguay,Uruguay,Isabelino Gradín,55.0,False,False
2,1916-07-02,Chile,Uruguay,Uruguay,Isabelino Gradín,70.0,False,False
3,1916-07-02,Chile,Uruguay,Uruguay,José Piendibene,75.0,False,False
4,1916-07-06,Argentina,Chile,Argentina,Alberto Ohaco,2.0,False,False


In [4]:
raw_results.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False


In [5]:
raw_shootouts.head()

,date,home_team,away_team,winner,first_shooter
0,1967-08-22,India,Taiwan,Taiwan,NaN
1,1971-11-14,South Korea,Vietnam Republic,South Korea,NaN
2,1972-05-07,South Korea,Iraq,Iraq,NaN
3,1972-05-17,Thailand,South Korea,South Korea,NaN
4,1972-05-19,Thailand,Cambodia,Thailand,NaN


### How many rows and how many columns?

We will alculate the number of rows and columns of three DataFrames `raw_goal_scorers`, `raw_results` and `raw_shootouts`. Then, store it in the variable `shape_goal_scorers`, `shape_results` and `shape_shootouts`

In [6]:
# YOUR CODE HERE
shape_goal_scorers = raw_goal_scorers.shape
shape_results = raw_results.shape
shape_shootouts = raw_shootouts.shape

In [7]:
print(f"Goal scorers shape: {shape_goal_scorers}")
print(f"Results shape: {shape_results}")
print(f"Shootouts shape: {shape_shootouts}")

Goal scorers shape: (44362, 8)
Results shape: (47917, 9)
Shootouts shape: (645, 5)


### What does each line mean? Does it matter if the lines have different meanings?

- Tất cả các dòng trong các file CSV này đều mô tả các trận đấu bóng đá từ các khía cạnh khác nhau: bàn thắng ghi được, kết quả trận đấu và các trận đấu có loạt sút penalty. Mỗi dòng cung cấp các thông tin chi tiết giúp ta phân tích và theo dõi các trận đấu, cầu thủ ghi bàn, tỉ số, và những chi tiết đặc biệt như sút penalty.
- Việc các dòng trong mỗi file phải có ý nghĩa và cấu trúc nhất quán theo từng cột là rất quan trọng. Điều này đảm bảo tính toàn vẹn của dữ liệu, tránh các lỗi phân tích và cho phép xử lý dữ liệu một cách suôn sẻ. Nếu dữ liệu trong một file bị trộn lẫn giữa các loại thông tin khác nhau, nó sẽ làm mất đi mục đích của dữ liệu và dẫn đến các kết luận sai lệch hoặc phân tích không chính xác.

### Does the raw data have duplicate rows?

We will calculate the number of rows with duplicate indexes and store it in the variable `goal_scorers_duplicated` and `results_duplicated` and `shootouts_duplicated`.

In [8]:
goal_scorers_duplicated = len(raw_goal_scorers[raw_goal_scorers.duplicated()])
results_duplicated = len(raw_results[raw_results.duplicated()])
shootouts_duplicated = len(raw_shootouts[raw_shootouts.duplicated()])

In [9]:
def check_duplicates(file_name, duplicated_count):
    if duplicated_count == 0:
        print(f"{file_name} data have no duplicated line.!")
    else:
        ext = "lines" if duplicated_count > 1 else "line"
        print(f"{file_name} data have {duplicated_count} duplicated {ext}!")

# Kiểm tra từng file
check_duplicates("Goal scorers", goal_scorers_duplicated)
check_duplicates("Results", results_duplicated)
check_duplicates("Shootouts", shootouts_duplicated)

Goal scorers data have 82 duplicated lines!
Results data have no duplicated line.!
Shootouts data have no duplicated line.!


In [10]:
#de-deduplicate Goal scorers data.
raw_goal_scorers.drop_duplicates(keep=False, inplace=True)

### What does each column mean?

#### Báo cáo dữ liệu bóng đá

Dưới đây là các bảng mô tả dữ liệu từ ba tệp CSV: **Goal Scorers**, **Results** và **Shootouts**.

<table style="width:48%; float:left; padding-right:10px; border: 1px solid black; border-collapse: collapse;">
  <caption><strong>Goal Scorers</strong></caption>
  <tr>
    <th style="border: 1px solid black;">Column</th>
    <th style="border: 1px solid black;">Description</th>
  </tr>
  <tr>
    <td style="border: 1px solid black;">date</td>
    <td style="border: 1px solid black;">Ngày diễn ra trận đấu.</td>
  </tr>
  <tr>
    <td style="border: 1px solid black;">home_team</td>
    <td style="border: 1px solid black;">Tên đội chủ nhà.</td>
  </tr>
  <tr>
    <td style="border: 1px solid black;">away_team</td>
    <td style="border: 1px solid black;">Tên đội khách.</td>
  </tr>
  <tr>
    <td style="border: 1px solid black;">team</td>
    <td style="border: 1px solid black;">Tên đội ghi bàn thắng.</td>
  </tr>
  <tr>
    <td style="border: 1px solid black;">scorer</td>
    <td style="border: 1px solid black;">Tên của cầu thủ ghi bàn.</td>
  </tr>
  <tr>
    <td style="border: 1px solid black;">minute</td>
    <td style="border: 1px solid black;">Phút ghi bàn.</td>
  </tr>
  <tr>
    <td style="border: 1px solid black;">own_goal</td>
    <td style="border: 1px solid black;">Liệu bàn thắng có phải là phản lưới nhà không.</td>
  </tr>
  <tr>
    <td style="border: 1px solid black;">penalty</td>
    <td style="border: 1px solid black;">Liệu bàn thắng có phải là quả phạt đền không.</td>
  </tr>
</table>

<table style="width:48%; float:left; border: 1px solid black; border-collapse: collapse;">
  <caption><strong>Results</strong></caption>
  <tr>
    <th style="border: 1px solid black;">Column</th>
    <th style="border: 1px solid black;">Description</th>
  </tr>
  <tr>
    <td style="border: 1px solid black;">date</td>
    <td style="border: 1px solid black;">Ngày diễn ra trận đấu.</td>
  </tr>
  <tr>
    <td style="border: 1px solid black;">home_team</td>
    <td style="border: 1px solid black;">Tên đội chủ nhà.</td>
  </tr>
  <tr>
    <td style="border: 1px solid black;">away_team</td>
    <td style="border: 1px solid black;">Tên đội khách.</td>
  </tr>
  <tr>
    <td style="border: 1px solid black;">home_score</td>
    <td style="border: 1px solid black;">Tỷ số toàn trận của đội chủ nhà bao gồm cả hiệp phụ, không bao gồm loạt sút penalty.</td>
  </tr>
  <tr>
    <td style="border: 1px solid black;">away_score</td>
    <td style="border: 1px solid black;">Tỷ số toàn trận của đội khách bao gồm cả hiệp phụ, không bao gồm loạt sút penalty.</td>
  </tr>
  <tr>
    <td style="border: 1px solid black;">tournament</td>
    <td style="border: 1px solid black;">Tên giải đấu.</td>
  </tr>
  <tr>
    <td style="border: 1px solid black;">city</td>
    <td style="border: 1px solid black;">Thành phố nơi diễn ra trận đấu.</td>
  </tr>
  <tr>
    <td style="border: 1px solid black;">country</td>
    <td style="border: 1px solid black;">Quốc gia nơi diễn ra trận đấu.</td>
  </tr>
  <tr>
    <td style="border: 1px solid black;">neutral</td>
    <td style="border: 1px solid black;">Cột True/False cho biết trận đấu có diễn ra tại sân trung lập (không phải sân nhà của đội nào) hay không.</td>
  </tr>
</table>

<br style="clear: both;">

<table style="width:48%; float:left; padding-right:10px; border: 1px solid black; border-collapse: collapse;">
  <caption><strong>Shootouts</strong></caption>
  <tr>
    <th style="border: 1px solid black;">Column</th>
    <th style="border: 1px solid black;">Description</th>
  </tr>
  <tr>
    <td style="border: 1px solid black;">date</td>
    <td style="border: 1px solid black;">Ngày diễn ra trận đấu.</td>
  </tr>
  <tr>
    <td style="border: 1px solid black;">home_team</td>
    <td style="border: 1px solid black;">Tên đội chủ nhà.</td>
  </tr>
  <tr>
    <td style="border: 1px solid black;">away_team</td>
    <td style="border: 1px solid black;">Tên đội khách.</td>
  </tr>
  <tr>
    <td style="border: 1px solid black;">winner</td>
    <td style="border: 1px solid black;">Tên đội thắng cuộc trong loạt sút penalty.</td>
  </tr>
  <tr>
    <td style="border: 1px solid black;">first_shooter</td>
    <td style="border: 1px solid black;">Tên đội thực hiện cú sút penalty đầu tiên.</td>
  </tr>
</table>


### What data type does each column currently have? Are there any columns whose data types are not suitable for further processing?

We will calculate the data type (dtype) of each column in DataFrame `raw_goal_scorers`, `raw_results`, `raw_shootouts` and save the result into Series `dtypes_goal_scorers`, `dtypes_results`, `dtypes_shootouts` (This Series has the index as the column name).

In [29]:
dtypes_dicts = {}

def get_dtype(raw_df):
    dtypes_dict = {}
    for column in raw_df.columns:
        dtypes_dict[column] = raw_df[column].dtype
    return dtypes_dict

# Tạo các dataframe và lưu lại dtypes vào dictionary
dtypes_dicts['goal_scorers'] = get_dtype(raw_goal_scorers)
dtypes_dicts['results'] = get_dtype(raw_results)
dtypes_dicts['shootouts'] = get_dtype(raw_shootouts)

# Chuyển các dictionary thành DataFrame
dtypes_goal_scorers = pd.DataFrame(dtypes_dicts['goal_scorers'].items(), columns=['Column Name', 'Data Type']).set_index('Column Name')
dtypes_results = pd.DataFrame(dtypes_dicts['results'].items(), columns=['Column Name', 'Data Type']).set_index('Column Name')
dtypes_shootouts = pd.DataFrame(dtypes_dicts['shootouts'].items(), columns=['Column Name', 'Data Type']).set_index('Column Name')

In [30]:
dtypes_goal_scorers

,Data Type
Column Name,
date,object
home_team,object
away_team,object
team,object
scorer,object
minute,float64
own_goal,bool
penalty,bool


In [31]:
dtypes_results

,Data Type
Column Name,
date,object
home_team,object
away_team,object
home_score,int64
away_score,int64
tournament,object
city,object
country,object
neutral,bool


In [32]:
dtypes_shootouts

,Data Type
Column Name,
date,object
home_team,object
away_team,object
winner,object
first_shooter,object


### For each column with numeric data type, how are the values distributed? 

For columns with numeric data types, you will calculate:
- Percentage (from 0 to 100) of missing values
- The min
- The median
- The max

You will save the results to a DataFrame `num_col_info_df`, where:
- The names of the columns are the names of the numeric columns in `raw_df`
- Names of rows: "min", "median" , "max" , "Percentage"



In [13]:
# YOUR CODE HERE

### For each categorical column, how are the values distributed?

For columns with non-numeric data types, you calculate:
- Percentage (from 0 to 100) of missing values
- Number of values (the values here are different values and we do not consider missing values): with columns whose type is categorical, it is a set with a finite number of categories. Directly counting the number of values in these columns doesn't make much sense, so it's better to count the number of elements of all types. (Số lượng các giá trị (các giá trị ở đây là các giá trị khác nhau và ta không xét giá trị thiếu): với cột mà có kiểu là categorical, nó là một tập hợp có số lượng hữu hạn các loại. Việc đếm trực tiếp số lượng các giá trị trong những cột này không có nhiều ý nghĩa, nên tốt hơn hết là mà sẽ đếm số lượng phần tử các loại.)
- The percentage (from 0 to 100) of each value is sorted by decreasing percentage (we do not consider missing values, the ratio is the ratio compared to the number of non-missing values): you use a dictionary to store , key is the value, value is the percentage; With the column corresponding to each type, the method is similar to above.

You will save the results to DataFrame `cat_col_info_df`, where:
- The names of the columns are the names of the non-numeric columns in `raw_df`
- The names of the lines are: "missing_ratio", "num_values", "value_ratios"

In [14]:
# YOUR CODE HERE

---

### Preprocessing + analyzing data to answer each question

**Question is:**  câu hỏi của mình nha

**Answering this question will**: lợi ích khi trả lời câu hỏi này

**How we answer this question**: ý tưởng làm câu hỏi này và nói cách làm

### Pre-processing

In [15]:
# YOUR CODE HERE (OPTION) 

### Analyzing to get the answer

In [16]:
# YOUR CODE HERE

### Your conclusion

YOUR CODE HERE